
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Model Boltzmanna</big></big></big></big></big>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [21]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

#from bokeh.io import gridplot, output_file, show
#from bokeh.plotting import figure, output_notebook
#from bkcharts import Scatter

import random

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="nn_figures/" width="100%">

# Maszyna Boltzmanna
1. składa się z
  * warstwy neuronów ___widzialnych___ $x$
  * warstwy neuronów ___ukrytych___ $h$
  * wag pomiędzy 
    * warstwami widzialną i ukrytą $W$
    * neuronami warstwy widzialnej $U$
    * neuronami warstwy ukrytej $V$
  * __wszystkie__ wagi są __symetryczne__
2. dla modelu określona jest energia
$$E(x,h)=-b^Tx - c^Th -h^TWx -x^TUx - h^TVh$$
  * dla prostoty wszędzie dalej będziemy określali 
    * wszystkie wagi przez $W$
    * wszystkie neurony przez $x$
  * oraz opuszczali temeperaturę $T$
3. prawdopodobieństwo stanu $x$ jest określone przez
$$P(x\mid W)=\frac{1}{Z}\,\exp\left(\frac{-E(x;W)}{k_B\,T}\right)$$
gdzie $Z$ jest __sumą statystyczną__ (partition function)
$$Z(W)=\sum_{x}\exp\left(-\frac{E(x;W)}{k_B\,T}\right)$$
  * stała Boltzmanna $k_B$ jest dana w Joulach na Kelvin
  * jeśli temperaturę określimy w Kelvinach, to energia jest bezjednostkowa
  * $Z(W)$ jest sumą energii po __wszystkich__ możliwych stanach
  * stany o __niskiej__ energii mają __wyższe__ prawdopodobieństwo
  * __zmniejszanie__ temperatury powoduje skupianie się prawdopodobieństwa na małym podzbiorze stanów o niskiej energii
    * osiągnie stan stabilny
    * __symulowane wyżarzanie__ (simmulated annealing)
      * wyższa tamperatura pozwala na łatwiejsze losowe przeskakiwanie do stanów o wyższej energii, co przyspiesza uczenie
      * trudne procedury prawidłowego schematu obniżania temperatury
      * temperatura musi spadać __nie szybciej__ niż logarytmicznie, co daje gwarancję zbieżności do minimum
      * zwykle bardzo powolne __wykładnicze__ obniżanie temperatury, np. $$T_k=\alpha\,T_{k-1}$$ dla $\alpha\in(0.8, 0.99)$
      * zakończenie gdy przez określoną liczbę kroków żadna propozycja nie jest akceptowana
4. funkcja log-likelihood układu
$$\begin{align}L(\theta)&=\log\prod_{x\in\,Data}P(X=x)\\&=\sum_{x\in\,Data}\log\,P(X=x)\end{align}$$
5. neurony widzialne i ukryte __różnią__ się
  * neurony widzialne dostają zewnętrzny sygnał (mają ustalane aktywacje)
  * neurony ukryte mają obliczane aktywacje na podstawie widzialnych
6. po obliczeniu ukrytych aktywacji, obliczane są widzialne i warstwami na zmianę aż do osiągnięcia stanu __stabilnego__
7. Przeliczając log-likelihood biorąc pod uwagę definicję energii dla stanu dostajemy
$$\log\left[\prod_{n=1}^NP(x^{(n)})\right]=\sum_{n=1}^N\left[\frac{1}{2}x^{(n)^T}Wx^{(n)}-\log\,Z(W)\right]$$
8. dla minimalizacji log-likelihood obliczamy pochodną po wagach
$$\begin{align}
\frac{\partial}{\partial w_{ij}}\log\,Z(W)&=\sum_xx_ix_jP(x\mid W)\\
&=\langle\,x_i\,x_j\rangle_{P(x\mid W)}\\
\frac{\partial}{\partial w_{ij}}\log\,P(\{x^{(n)}\}_{n=1}^N\mid\,W)&=\sum_{n=1}^N\left[x_i^{(n)}x_j^{(n)}-\langle\,x_i\,x_j\rangle_{P(x\mid W)}\right]\\
&=N\left[\langle\,x_ix_j\rangle_{Data}-\langle\,x_i\,x_j\rangle_{P(x\mid W)}\right]
\end{align}$$
  * pierwszy składnik ostatniego wyrażenia
    * iloczyny skalarne pomiedzy neuronami dla wartosci wylosowanych (wysamplowanych) z prawdziwego rozkładu danych $Data$
    * to tzw. faza __świadomości__, faza __dodatnia__
  * drugi składnik
    * iloczyny skalarne dla wszystkich możliwych stanów
    * obejmuje także stany warstwy ukrytej
    * to tzw. faza __marzeń sennych__, faza __ujemna__

## Restricted Boltzmann Machine RBM
1. uczenie maszyny Boltzmanna w cyklu fazy dodatniej-ujemnej jest __bardzo__ powolne
2. ograniczenie maszyny Boltzmanna przez usunięcie połączeń wewnątrz warstw widzialnej i ukrytej
$$\begin{align}
E(x,h)&=-b^{T}x-c^{T}h-h^{T}Wx\\
P(h\mid x)&=\prod_{i}P(h_{i}\mid x)\\
P(x\mid h)&=\prod_{i}P(x_{j}\mid h)\\
P(h_{i}=1\mid x)&=\frac{exp(c_{i}+W_{i}x)}{1+exp(c_{i}+W_{i}x)}=sigm(c_{i}+W_{i}x)
\end{align}$$
3. neurony w poszczególnych warstwach stają się __niezależne__ od siebie i można je liczyć niezależnie
  * aktywacje dla ukrytych w zależności od widzialnych z ustawioną (clamped) aktywacją $P(h\mid x)$
  * aktywacje widzialnych na podstawie ukrytych można liczyć niezależnie
4. uczenie
  1. dla każdej $w_{ij}$ korelacja $\langle\,v_i^0h_j^0\rangle$ dla zadanych widzialnych
  2. próbkowanie Gibbsa aż do osiągnięcia stanu stabilnego i korelacji $\langle\,v_i^\infty\,h_j^\infty\rangle$
  3. gradient
  $$\frac{\partial\,\log\,p(v^0)}{\partial\,w_{ij}}=\langle\,v_i^0h_j^0\rangle-\langle\,v_i^\infty\,h_j^\infty\rangle$$
5. Hinton pokazał, że __wystarczy__ przeprowadzić __dwa__ cykle przetwarzania
  1. obliczanie aktywacji ukrytych wprzód
  2. obliczenie wejściowych na podstawie ukrytych
  
  by uzyskać wystarczające przybliżenie całego procesu

## Deep Boltzmann Machine
1. RBM __nie jest__ wystarczającym modelem
2. uczenie głębokiego modelu w oczywisty sposób napotkało na trudności
  * trudność z definicją przetwarzania wprzód-wstecz
  * zanikanie sygnału gradientu
    * wykorzystane aktywacje sigmoidalne
  * problem z definicją uczenia nadzorowanego
3. Hinton (i współpracownicy) pokazał algorytm __zachłannego__ uczenia modelu
  * uczenie nadzorowane ze wstępnym etapem uczenia nienadzorowanego
  * rozpoczynamy od prostej RBM złożonej z warstw $x, h_1$
  * ta jest uczona do stanu względnej stabilności
  * dodawana jest nowa warstwa tworząca na szczycie nową RBM
  * znowu uczenie jej do stanu stabilności
    * ustalenie likelihood dla $P(h_2\mid h_1)$
  * tak do osiągniecia ustalonej liczby warstw
  * to kończy wstępny etap uczenia __nienadzorowanego__
  * na szczycie warstwa z uczeniem nadzorowanym
  * uczenie tej warstwy
  * następnie __tuning__ całego modelu w trybie nadzorowanym
3. Salakhutdinov zaproponował rozszerzenie
  * podobnie zachłanne uczenie
  * w trakcie dodawania nowych warstw i uczenia $P(h_k\mid h_{k-1})$
    * uczenie $P(h_{k-1}\mid h_k)$ i wstecz
  * bardzo złożony algorytm nawet mimo kilku sprytnych trików
  